# 미션: 글로벌 스포츠 이벤트 실시간 트위터 감정 모니터링

당신은 세계적인 스포츠 대회 조직위원회 SNS 분석팀의 신규 AI 엔지니어입니다. 대회 기간 중 팬들이 트위터에 올리는 다양한 반응을 실시간으로 분석해, 긍정·부정 감정을 파악하고 주요 이슈를 빠르게 리포트해야 합니다. 이를 위해 아래 과제를 수행하세요.

---
**과제: SNS 감정 분석 파이프라인 구축**

다음 단계별 과제를 Python(Jupyter Notebook)으로 구현하여 제출하세요. 각 단계별로 코드와 간단한 설명(markdown)을 함께 달아야 합니다.


## 1. 데이터 로드
- `nltk.download('twitter_samples')`로 리소스 설치
- 긍정 트윗(`positive_tweets.json`) 1,000개, 부정 트윗(`negative_tweets.json`) 1,000개 로드

In [115]:
# 1. 데이터 로드 단계 코드 작성
import nltk
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples

pos_tweets = twitter_samples.strings('positive_tweets.json')
neg_tweets = twitter_samples.strings('negative_tweets.json')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\wjdgn\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


## 2. 텍스트 전처리 (Cleaning & Normalization)
- 소문자화: 모든 문자를 소문자로 변환
- 불필요 문자 제거: URL, 이모지, 특수문자, 숫자 등을 정규표현식으로 제거
- 공백 정리: 연속된 공백을 하나로 축소

In [116]:
import re

# 2. 텍스트 전처리 (Cleaning & Normalization) 단계 코드 작성
def clean_tweet(text):
    # 소문자화
    text = text.lower()
    # URL 제거
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    # 이모지 및 특수문자 제거 (유니코드 범위)
    text = re.sub(r'[\U00010000-\U0010ffff]', '', text)
    # 특수문자, 숫자, 언급, 해시태그, &amp; 등 제거
    text = re.sub(r'[@#]\w+|&\w+;|[^a-z\s]', ' ', text)
    # 숫자 제거
    text = re.sub(r'\d+', '', text)
    # 다중 공백을 단일 공백으로
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 긍정/부정 트윗 전처리
pos_clean = [clean_tweet(t) for t in pos_tweets]
neg_clean = [clean_tweet(t) for t in neg_tweets]

# 예시 출력
print("Before:", pos_tweets[0])
print("After :", pos_clean[0])

Before: #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
After : for being top engaged members in my community this week


## 3. 토큰화 (Sentence & Word Tokenization)
- `sent_tokenize`로 샘플 트윗 5개를 문장 단위로 분리
- `word_tokenize`로 각 문장을 단어 토큰화하여 첫 10개 토큰 출력

In [117]:
# 3. 토큰화 (Sentence & Word Tokenization) 단계 코드 작성
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

# 샘플 트윗 5개 선택 (전처리 전 원문 사용)
sample_tweets = pos_clean[:5]

# 각 트윗을 문장 단위로 분리
for i, tweet in enumerate(sample_tweets):
    print(f"\n[{i+1}번 트윗]")
    sentences = sent_tokenize(tweet)
    print("문장 분리:", sentences)
    # 각 문장을 단어 토큰화하여 첫 10개 토큰 출력
    for j, sent in enumerate(sentences):
        tokens = word_tokenize(sent)
        print(f"  문장 {j+1} 토큰(최대 10개):", tokens[:10])


[1번 트윗]
문장 분리: ['for being top engaged members in my community this week']
  문장 1 토큰(최대 10개): ['for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week']

[2번 트윗]
문장 분리: ['hey james how odd please call our contact centre on and we will be able to assist you many thanks']
  문장 1 토큰(최대 10개): ['hey', 'james', 'how', 'odd', 'please', 'call', 'our', 'contact', 'centre', 'on']

[3번 트윗]
문장 분리: ['we had a listen last night as you bleed is an amazing track when are you in scotland']
  문장 1 토큰(최대 10개): ['we', 'had', 'a', 'listen', 'last', 'night', 'as', 'you', 'bleed', 'is']

[4번 트윗]
문장 분리: ['congrats']
  문장 1 토큰(최대 10개): ['congrats']

[5번 트윗]
문장 분리: ['yeaaaah yippppy my accnt verified rqst has succeed got a blue tick mark on my fb profile in days']
  문장 1 토큰(최대 10개): ['yeaaaah', 'yippppy', 'my', 'accnt', 'verified', 'rqst', 'has', 'succeed', 'got', 'a']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wjdgn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 4. 불용어 제거 (Stopwords)
- NLTK 영어 불용어 목록(`stopwords.words('english')`) 적용
- 추가로 직접 정의한 ‘비속어·의미 없는 단어’ 5개 포함하여 제거

In [ ]:
# 4. 불용어 제거 (Stopwords) 단계 코드 작성
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# wonna gonna 를 want to , going to 로 변환
def replace_wanna_gonna(text):
    text = re.sub(r'\bwanna\b', 'want to', text)
    text = re.sub(r'\bgonna\b', 'going to', text)
    return text
# 긍정/부정 트윗에서 wanna, gonna 변환
pos_clean = [replace_wanna_gonna(tweet) for tweet in pos_clean]
neg_clean = [replace_wanna_gonna(tweet) for tweet in neg_clean]
# 불용어 제거 함수
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]
# 긍정/부정 트윗에서 불용어 제거
pos_tokens = [word_tokenize(tweet) for tweet in pos_clean]
neg_tokens = [word_tokenize(tweet) for tweet in neg_clean]
pos_tokens = [remove_stopwords(tokens) for tokens in pos_tokens]
neg_tokens = [remove_stopwords(tokens) for tokens in neg_tokens]
# 예시 출력
print("\n긍정 트윗 불용어 제거 예시:")
for i in range(3):
    print(f"원문: {pos_clean[i]}")
    print("불용어 제거:", pos_tokens[i])

custom_stopwords = ['top', 'listen', 'last']
# 사용자 정의 불용어 추가
stop_words.update(custom_stopwords)
# 사용자 정의 불용어 제거
def remove_custom_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]
# 긍정/부정 트윗에서 사용자 정의 불용어 제거
pos_tokens = [remove_custom_stopwords(tokens) for tokens in pos_tokens]
neg_tokens = [remove_custom_stopwords(tokens) for tokens in neg_tokens]
# 예시 출력
print("\n긍정 트윗 커스터 불용어 제거 예시:")
for i in range(3):
    print(f"원문: {pos_clean[i]}")
    print("불용어 제거:", pos_tokens[i])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wjdgn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



긍정 트윗 불용어 제거 예시:
원문: for being top engaged members in my community this week
불용어 제거: ['top', 'engaged', 'members', 'community', 'week']
원문: hey james how odd please call our contact centre on and we will be able to assist you many thanks
불용어 제거: ['hey', 'james', 'odd', 'please', 'call', 'contact', 'centre', 'able', 'assist', 'many', 'thanks']
원문: we had a listen last night as you bleed is an amazing track when are you in scotland
불용어 제거: ['listen', 'last', 'night', 'bleed', 'amazing', 'track', 'scotland']

긍정 트윗 커스터 불용어 제거 예시:
원문: for being top engaged members in my community this week
불용어 제거: ['engaged', 'members', 'community', 'week']
원문: hey james how odd please call our contact centre on and we will be able to assist you many thanks
불용어 제거: ['hey', 'james', 'odd', 'please', 'call', 'contact', 'centre', 'able', 'assist', 'many', 'thanks']
원문: we had a listen last night as you bleed is an amazing track when are you in scotland
불용어 제거: ['night', 'bleed', 'amazing', 'track', 'scotland

## 5. 단어 사전 구축 (Vocabulary)
- 전처리된 전체 토큰에서 단어 빈도수 계산
- 상위 5,000개 단어에 고유 인덱스 부여하여 사전 생성
- 사전 크기, 상위 20개 단어 및 각 단어의 빈도를 표로 정리

In [119]:
# 5. 단어 사전 구축 (Vocabulary) 단계 코드 작성
from collections import Counter
# 전체 트윗에서 단어 사전 구축
all_tweets = pos_tokens + neg_tokens

# 상위 5000개 단어에 고유 인덱스 부여하여 사전 구축축
vocab = Counter(word for tokens in all_tweets for word in tokens)
vocab = vocab.most_common(5000)
# 단어 빈도수 계산
word_freq = Counter(word for tokens in all_tweets for word in tokens)
# 상위 20개 단어 출력
print("\n상위 20개 단어 및 빈도수:")
for word, freq in word_freq.most_common(20):
    print(f"{word}: {freq}")


상위 20개 단어 및 빈도수:
thanks: 470
follow: 446
u: 441
like: 425
want: 416
love: 405
please: 370
get: 349
good: 334
day: 308
back: 283
know: 278
thank: 277
see: 275
one: 272
miss: 259
time: 254
going: 228
much: 228
today: 226


## 6. 정수 인코딩 & 패딩 (Integer Encoding & Padding)
- 각 트윗을 사전 인덱스로 변환하여 정수 시퀀스 생성
- 최대 길이 50, `padding='post'` 방식으로 패딩
- 패딩 전·후 한 문장 예시 출력

In [120]:
!pip install keras
!pip install tensorflow


In [121]:
# 6. 정수 인코딩 & 패딩 (Integer Encoding & Padding) 단계 코드 작성
# 각 트윗을 사전 인덱스로 변환하여 정수 시퀀스 생성
word_to_index = {word: i for i, (word, _) in enumerate(vocab)}
def encode_tweet(tokens):
    return [word_to_index[word] for word in tokens if word in word_to_index]
# 정수 인코딩
pos_encoded = [encode_tweet(tokens) for tokens in pos_tokens]
neg_encoded = [encode_tweet(tokens) for tokens in neg_tokens]

In [122]:
# 최대 길이 50, padding = 'post' 방식으로 패딩
from keras.preprocessing.sequence import pad_sequences
max_length = 50
pos_padded = pad_sequences(pos_encoded, maxlen=max_length, padding='post')
neg_padded = pad_sequences(neg_encoded, maxlen=max_length, padding='post')

# 패딩 전.후 문장 예시 출력
print("\n패딩 전/후 문장 예시:")
for i in range(3):
    print(f"원문: {pos_tweets[i]}")
    print("전처리 후:", pos_clean[i])
    print("패딩 전:", pos_encoded[i])
    print("패딩 후:", pos_padded[i])


패딩 전/후 문장 예시:
원문: #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
전처리 후: for being top engaged members in my community this week
패딩 전: [1011, 497, 270, 51]
패딩 후: [1011  497  270   51    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
원문: @Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!
전처리 후: hey james how odd please call our contact centre on and we will be able to assist you many thanks
패딩 전: [59, 615, 1974, 6, 183, 814, 1975, 271, 4088, 120, 0]
패딩 후: [  59  615 1974    6  183  814 1975  271 4088  120    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
   

## 7. 벡터화 (Vectorization)
- Bag-of-Words 및 TF–IDF 방식 적용
- 전체 코퍼스에 대해 문서–단어 희소 행렬 생성
- 희소 행렬 형태와 밀집 배열(첫 5개 문서) 비교 출력
- `sklearn.feature_extraction.text` 모듈 사용

In [123]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# 1. 토큰 리스트를 다시 문자열로 결합
docs = [' '.join(tokens) for tokens in all_tweets]

# 2. Bag-of-Words 벡터화
bow_vectorizer = CountVectorizer(max_features=20)
X_bow = bow_vectorizer.fit_transform(docs)

# 3. TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(max_features=20)
X_tfidf = tfidf_vectorizer.fit_transform(docs)

# 4. 출력
print("Bag-of-Words 희소 행렬 형태:", X_bow.shape, type(X_bow))
print("TF-IDF 희소 행렬 형태:", X_tfidf.shape, type(X_tfidf))

print("\nBag-of-Words (첫 5개 문서, 밀집 배열):")
print(X_bow[:5].toarray())

print("\nTF-IDF (첫 5개 문서, 밀집 배열):")
print(X_tfidf[:5].toarray())


Bag-of-Words 희소 행렬 형태: (10000, 20) <class 'scipy.sparse._csr.csr_matrix'>
TF-IDF 희소 행렬 형태: (10000, 20) <class 'scipy.sparse._csr.csr_matrix'>

Bag-of-Words (첫 5개 문서, 밀집 배열):
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

TF-IDF (첫 5개 문서, 밀집 배열):
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.73662503 0.         0.         0.67630138 0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.      

In [124]:
# 벡터화 후 실제 단어 확인
print("\n벡터화된 단어 수:", len(bow_vectorizer.vocabulary_))
print("상위 단어 10개:", list(bow_vectorizer.vocabulary_.keys())[:10])



벡터화된 단어 수: 20
상위 단어 10개: ['please', 'thanks', 'one', 'like', 'time', 'go', 'love', 'follow', 'back', 'know']


## 8. Bigram 모델 (간단 예제)
아래 아주 단순한 코퍼스로 Bigram 확률을 직접 계산해 보세요:



In [125]:
# 8. Bigram 모델 (간단 예제) 단계 코드 작성
# 이 코드는 전처리가 제대로 안되어 실패하는 코드임

from collections import Counter

corpus = ["나는 학교에 간다", "나는 집에 간다", "나는 학교에 간다"]
tokenized = [s.split() for s in corpus]

unigram = Counter()
bigram  = Counter()
for toks in tokenized:
    unigram.update(toks)
    bigram.update(zip(toks[:-1], toks[1:]))

prev = "나는"
p_school = bigram[(prev,"학교")] / unigram[prev]
p_home   = bigram[(prev,"집")]   / unigram[prev]
print(f"P('학교'|'나는') = {p_school}, P('집'|'나는') = {p_home:.2f}")


P('학교'|'나는') = 0.0, P('집'|'나는') = 0.00


In [126]:
# 8. Bigram 모델 (간단 예제) 단계 코드 작성

# konlpy 설치가 필요하다면
# !pip install konlpy

from konlpy.tag import Okt
from collections import Counter

# 1) 형태소 분석기 초기화
okt = Okt()

# 2) 원문 코퍼스
corpus = ["나는 학교에 간다", "나는 집에 간다", "나는 학교에 간다"]

# 3) 형태소 분석 및 불필요 품사 제거
tokenized = []
for sent in corpus:
    pos = okt.pos(sent, norm=True, stem=True)
    # Josa(조사), Eomi(어미), Punctuation(구두점) 제외
    toks = [word for word, tag in pos if tag not in ('Josa','Eomi','Punctuation')]
    tokenized.append(toks)

print("토큰화 결과:", tokenized)
# → [['나','학교','간다'], ['나','집','간다'], ['나','학교','간다']]

# 4) Unigram / Bigram 카운트
unigram = Counter()
bigram  = Counter()
for toks in tokenized:
    unigram.update(toks)
    bigram.update(zip(toks[:-1], toks[1:]))

# 5) 조건부 확률 계산
prev = '나'
p_school = bigram[(prev, '학교')] / unigram[prev]
p_home   = bigram[(prev, '집')]   / unigram[prev]

print(f"P('학교'|'나') = {p_school:.2f}, P('집'|'나') = {p_home:.2f}")
# → P('학교'|'나') = 0.67, P('집'|'나') = 0.33


토큰화 결과: [['나', '학교', '간다'], ['나', '집', '간다'], ['나', '학교', '간다']]
P('학교'|'나') = 0.67, P('집'|'나') = 0.33


## 9. 트위터 데이터셋을 활용한 Bigram 모델 실습
NLTK `twitter_samples`의 긍정·부정 토큰을 이용해 다음을 수행하세요:

- **Unigram / Bigram 카운트**

```python
from collections import Counter

pos_tokens = [...]  # 4번 단계 결과 사용
neg_tokens = [...]  # 4번 단계 결과 사용

unigram_pos = Counter(pos_tokens)
bigram_pos  = Counter(zip(pos_tokens[:-1], pos_tokens[1:]))
unigram_neg = Counter(neg_tokens)
bigram_neg  = Counter(zip(neg_tokens[:-1], neg_tokens[1:]))
```

- **조건부 확률 계산**

```python
p_pos = bigram_pos[('i','love')] / unigram_pos['i']
p_neg = bigram_neg[('i','love')] / unigram_neg['i']
print(f"P(i→love|pos) = {p_pos:.2f}, P(i→love|neg) = {p_neg:.2f}")
```

- **상위 10개 Bigram 비교**

```python
print("Top10 positive:", bigram_pos.most_common(10))
print("Top10 negative:", bigram_neg.most_common(10))
```

In [127]:
from collections import Counter

# 1. Unigram / Bigram 카운트
# 모든 토큰을 하나의 리스트로 합침
flat_pos = [token for tweet in pos_tokens for token in tweet]
flat_neg = [token for tweet in neg_tokens for token in tweet]

unigram_pos = Counter(flat_pos)
unigram_neg = Counter(flat_neg)
bigram_pos = Counter(zip(flat_pos[:-1], flat_pos[1:]))
bigram_neg = Counter(zip(flat_neg[:-1], flat_neg[1:]))

# 2. 조건부 확률 계산
p_pos = bigram_pos[('i', 'love')] / unigram_pos['i'] if unigram_pos['i'] > 0 else 0
p_neg = bigram_neg[('i', 'love')] / unigram_neg['i'] if unigram_neg['i'] > 0 else 0
print(f"P(i→love|pos) = {p_pos:.2f}, P(i→love|neg) = {p_neg:.2f}")

# 3. 상위 10개 Bigram 비교
print("Top10 positive:", bigram_pos.most_common(10))
print("Top10 negative:", bigram_neg.most_common(10))

P(i→love|pos) = 0.00, P(i→love|neg) = 0.00
Top10 positive: [(('follow', 'follow'), 64), (('follow', 'u'), 62), (('u', 'back'), 62), (('arrived', 'new'), 60), (('unfollowers', 'via'), 60), (('happy', 'friday'), 50), (('happy', 'birthday'), 50), (('love', 'x'), 49), (('lot', 'see'), 45), (('hi', 'bam'), 44)]
Top10 negative: [(('follow', 'please'), 73), (('thanks', 'please'), 52), (('please', 'followed'), 52), (('followed', 'thanks'), 51), (('please', 'follow'), 45), (('love', 'much'), 43), (('want', 'go'), 42), (('much', 'beli'), 35), (('beli', 'eve'), 35), (('eve', 'wi'), 35)]


In [128]:
# Laplace 스무딩(α=1) 적용된 Bigram 조건부 확률 계산

# α 설정
alpha = 1

# 어휘 크기 계산 (positive, negative 각각)
V_pos = len(unigram_pos)
V_neg = len(unigram_neg)

# 스무딩 적용된 조건부 확률
p_pos = (bigram_pos[('i', 'love')] + alpha) / (unigram_pos['i'] + alpha * V_pos)
p_neg = (bigram_neg[('i', 'love')] + alpha) / (unigram_neg['i'] + alpha * V_neg)

print(f"Laplace smoothing 적용 후 P(i→love|pos) = {p_pos:.4f}, P(i→love|neg) = {p_neg:.4f}")

Laplace smoothing 적용 후 P(i→love|pos) = 0.0002, P(i→love|neg) = 0.0001


In [129]:
print("Top10 positive:", bigram_pos.most_common(10))
print("Top10 negative:", bigram_neg.most_common(10))


Top10 positive: [(('follow', 'follow'), 64), (('follow', 'u'), 62), (('u', 'back'), 62), (('arrived', 'new'), 60), (('unfollowers', 'via'), 60), (('happy', 'friday'), 50), (('happy', 'birthday'), 50), (('love', 'x'), 49), (('lot', 'see'), 45), (('hi', 'bam'), 44)]
Top10 negative: [(('follow', 'please'), 73), (('thanks', 'please'), 52), (('please', 'followed'), 52), (('followed', 'thanks'), 51), (('please', 'follow'), 45), (('love', 'much'), 43), (('want', 'go'), 42), (('much', 'beli'), 35), (('beli', 'eve'), 35), (('eve', 'wi'), 35)]


---
## 제출 형식

- 각 단계별 코드와 결과, 간단한 설명(markdown) 포함  
- 필요에 따라 시각화(matplotlib) 또는 표 활용  
- 최종 파일: Jupyter Notebook(.ipynb)으로 제출  

이 과제를 통해, 감정 분석의 전체 워크플로우(데이터 로드 → 전처리 → 토큰화/불용어 제거 → 사전 구축 → 인코딩·패딩 → 벡터화 → Bigram 모델 → 감정 분류)를 직접 구현·체험할 수 있습니다. 재미있게 도전해 보세요!
